In [ ]:
#Notatnik do tworzenia Glue Joba 


In [ ]:
%region eu-west-1
%profile superuser-test    
%iam_role arn:aws:iam::866525145675:role/LSM-DP-Genius-Rec-Glue-Service-Role-test
%glue_version 3.0
%number_of_workers 2
%worker_type G.1X
%idle_timeout 60
%session_id_prefix lsm-dp
%session_type ETL
%extra_jars s3://lsm-dp-utility-eu-test/jre/sqljdbc41.jar



In [ ]:
%%assume_role 
{'arn:aws:iam::866525145675:role/LSM-DP-Genius-Rec-Glue-Service-Role-test'}

In [ ]:
%%configure 
{
    "--extra-py-files": "s3://grsi-dp-glue-script-eu-test/scripts2.0/etl.zip"
}


In [ ]:
%%tags 
{
    "lm_app":"aws-dp", 
    "lm_app_env":"test", 
    "lm_sbu":"GS", 
    "lm_troux_uid":"B3439EB1-509F-4193-B1B3-2023D75CBD29", 
    "Name":"grsi-dp-int-glue-sesion-test", 
    "deployment_guid":"2f72a84f-6093-4813-8781-77ca784427b8",
    "organization_guid":"2326c977-2f8d-4d97-8f13-a7ca2fd5d5db"
}


In [ ]:

#from etl.shared import DataRegion
#from etl.util import S3Helper, GlueJobUtil, GlueLogger
#from etl.config import ConfigManager
#from etl.extraction.util import ColumnMappingDynamoDBWriter
#from etl.extraction.extractors import S3Extractor
#from etl.extraction.writers  import S3Writer
#from etl.extraction.dataflows import RegionalDataFlow

In [ ]:
import sys
from pathlib import Path
from typing import List




In [ ]:
#Pomocnicze Klasy z pliku util.py
import sys
from pathlib import Path
from typing import List

import boto3
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from awsglue.context import GlueContext # type: ignore

from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import *
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from pyspark.context import SparkContext
from pyspark.sql.functions import lit, col
from extract_utils import *


class GlueLogger:
    """
    wrapper class for the AWS Glue Logger to log any application-specific messages in the script
    that are sent in real time to the driver log stream.
    """

    def __init__(self, logger):
        self.logger = logger


    def __getattr__(self, k):
        """
        Dynamically return underlying Method for Glue Logger with String Representation of args
        """
        def wrapper(*args, **kwargs):
            return getattr(self.logger, k)(*[repr(a) for a in args], **kwargs)

        return wrapper

class S3Helper:

    def __init__(
        self,
        s3_bucket: str, 
    ) -> None:
        self.s3_bucket = s3_bucket
        self._s3_resource = boto3.resource("s3")

    def get_json_object(self, key, encoding="utf-8"):
        obj = self._s3_resource.Object(self.s3_bucket, key)
        return obj.get()["Body"].read().decode(encoding)
    
    def list_objects(self, s3_folder) -> List[str]:
        s3_folder = S3Helper.add_slash(s3_folder)
        bucket = self._s3_resource.Bucket(self.s3_bucket)
        
        files = []
        for obj in bucket.objects.filter(Prefix=s3_folder):
            files.append(obj.key)

        return files
    
    @classmethod
    def get_filename_from_path(cls, s3_path):
        return Path(s3_path).stem

    @classmethod
    def add_slash(cls, s):
        return s if s[-1] == "/" else s + "/"
    

class GlueJobUtil:

    @staticmethod
    def get_jdbc_mssql_conf_from_glue_connection(glue_context: GlueContext, connection_name: str) -> dict:
        
        glue_conn_properties = glue_context.extract_jdbc_conf(connection_name=connection_name)
        return {
            "url": glue_conn_properties.get("fullUrl"),
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
            "user": glue_conn_properties.get("user"),
            "password": glue_conn_properties.get("password"),
        }


    @staticmethod
    def get_ddb_column_mappings_table_name(table_prefix: str, aws_region: str, app_env: str) -> str:
        """
        :param table_prefix: Dynamo table name prefix
        :param aws_region: AWS region string e.g. 'us-east-1'
        :param app_env: Environment e.g. test
        :return: DynamoDB table for the target region and environment to store column mappings
        """
        return f"{table_prefix}-{GlueJobUtil.get_region_code(aws_region)}-{app_env}".lower()

    @staticmethod
    def get_assumed_role_arn(role_prefix: str, aws_region: str, account_number: str, app_env: str) -> str:
        """
        Derives the cross-account role name for the corresponding region to create clients and resources
        :param role_prefix: Role name prefix
        :param aws_region: AWS region string e.g. 'us-east-1'
        :param account_number: Account number of the region that the Role to assume is in
        :param app_env: Environment e.g. test
        :return: role arn
        """
        return f"arn:aws:iam::{account_number}:role/{role_prefix}-{GlueJobUtil.get_region_code(aws_region)}-{app_env}".lower()

    @staticmethod
    def get_ingestion_bucket_name(bucket_prefix: str, aws_region: str, app_env: str) -> str:
        """
        :param bucket_prefix: Ingestion bucket base name
        :param aws_region: AWS region string e.g. 'us-east-1'
        :param app_env: Environment e.g. test
        :return: ingestion bucket name for the target region and environment
        """
        return f"{bucket_prefix}-{GlueJobUtil.get_region_code(aws_region)}-{app_env}".lower()
    
    @staticmethod
    def get_region_code(aws_region) -> str:
        """
        :param region: AWS region string e.g. 'us-east-1'
        :return: Region code e.g. 'us'
        """
        region_code = aws_region[0:2]
        return region_code
    
    @staticmethod
    def get_assumed_region_session(
        app_env,
        assume_role_prefix,
        region,
        account_number,
    ) -> boto3.session.Session:
        """
        Assume the cross-account role and return a boto3 session for the corresponding region to create clients and resources
        :param account_number: Account number of the region that the Role to assume is in
        :param app_env: The Environment
        :param region: The AWS region, e.g. 'us-east-1' or 'eu-west-1'
        :param assume_role_prefix:  Role prefix to use  for assumption
        :return:
        """

        assume_role = GlueJobUtil.get_assumed_role_arn(assume_role_prefix, region, account_number, app_env)
        sts = boto3.client("sts")
        assume_response = sts.assume_role(
            RoleArn=assume_role, RoleSessionName="glueAssumeRole"
        )

        session = boto3.session.Session(
            region_name=region,
            aws_access_key_id=assume_response["Credentials"]["AccessKeyId"],
            aws_secret_access_key=assume_response["Credentials"]["SecretAccessKey"],
            aws_session_token=assume_response["Credentials"]["SessionToken"],
        )
        return session


    @staticmethod
    def get_hadoop_fs_kms_config_from_ssm(
        app_env, bucket, region, kms_key_ssm_param_prefix, boto_session
    ) -> list[tuple]:
        """
        Static function to use Boto3 to assume cross-account role and retrieve SSM Parameter
        :param boto_session:
        :param app_env: The Environment
        :param region: The AWS region, e.g. 'us-east-1' or 'eu-west-1'
        :param bucket: Bucket name for config
        :param kms_key_ssm_param_prefix: Name of the SSM Parameter containing the desired KMS ARN value
        :return: List of Spark config tuples for desired bucket and environment
        """
        region_code = GlueJobUtil.get_region_code(region)
        kms_key_ssm_param = "-".join([kms_key_ssm_param_prefix, region_code, app_env])

        ssm = boto_session.client("ssm")
        ssm_response = ssm.get_parameter(Name=kms_key_ssm_param, WithDecryption=True)

        return [
            (
                f"spark.hadoop.fs.s3a.bucket.{bucket}.server-side-encryption.key",
ssm_response["Parameter"]["Value"],
            ),
            (
                f"spark.hadoop.fs.s3a.{bucket}.server-side-encryption-algorithm",
                "SSE-KMS",
            ),
        ]
    
    
    @staticmethod
    def create_contexts(
        app_env: str,
        aws_region_account_mappings: dict,
        bucket_prefix: str,
        assume_role_prefix: str,
        kms_key_ssm_param_prefix: str,
        additional_spark_config: List[tuple] = []
    ):
        """
        Static Function to create SparkContext, GlueContext, SparkSession objects
        :return spark_context: SparkContext object
        :return glue_context: SparkContext object
        :return spark_session: SparkSession object
        """
        sparkConfig = [
            (
                "spark.hadoop.fs.s3a.server-side-encryption.key",
                "",
            ),
            (
                "spark.hadoop.fs.s3a.server-side-encryption-algorithm",
                "SSE-KMS",
            ),
        ]
        sparkConfig.extend(additional_spark_config)

        # Generate additional Spark config per bucket per region
        for aws_region, account_number in aws_region_account_mappings.items():
            assumed_session = GlueJobUtil.get_assumed_region_session(
                app_env=app_env,
                region=aws_region,
                assume_role_prefix=assume_role_prefix,
                account_number=account_number,
            )
            sparkConfig.extend(
                GlueJobUtil.get_hadoop_fs_kms_config_from_ssm(
                    app_env=app_env,
                    bucket=GlueJobUtil.get_ingestion_bucket_name(bucket_prefix, aws_region, app_env),
                    region=aws_region,
                    kms_key_ssm_param_prefix=kms_key_ssm_param_prefix,
                    boto_session=assumed_session,
                )
            )

        config = SparkConf().setAll(sparkConfig)
        spark_context = SparkContext.getOrCreate(config)
        glue_context = GlueContext(spark_context)
        spark_session = glue_context.spark_session
        spark_session.sql("set spark.sql.parquet.int96RebaseModeInWrite=CORRECTED")
        spark_session.sql("set spark.sql.caseSensitive=false")
        
        return spark_context, glue_context, spark_session

    
    @staticmethod
    def create_data_lake_contexts(
        app_env: str,
        aws_region_account_mappings: dict,
        bucket_prefix: str,
        assume_role_prefix: str,
        kms_key_ssm_param_prefix: str,
        datalake_cross_account_role: str,
        datalake_bucket: str
    ):
        """
        Static Function to create SparkContext, GlueContext, SparkSession objects
        with additional Spark config for Delta Lake packages & temp credentials
        :return spark_context: SparkContext object
        :return glue_context: SparkContext object
        :return spark_session: SparkSession object
        """

        sts = boto3.client("sts")
        datalake_session = sts.assume_role(RoleArn=datalake_cross_account_role, RoleSessionName="datalakeAssumeRole")
        datalake_credentials = datalake_session['Credentials']

        dataLakeConfig = [
            (
                "spark.sql.catalog.spark_catalog",
                "org.apache.spark.sql.delta.catalog.DeltaCatalog",
            ), 
            (
                "spark.delta.logStore.class",
                "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore",
            ), 
            (
                f"fs.s3a.bucket.{datalake_bucket}.access.key",
                datalake_credentials['AccessKeyId']
            ),
            (
                f"fs.s3a.bucket.{datalake_bucket}.secret.key",
                datalake_credentials['SecretAccessKey']
            ),
            (
                f"fs.s3a.bucket.{datalake_bucket}.session.token",
                datalake_credentials['SessionToken']
            ),
            (
                f"fs.s3a.bucket.{datalake_bucket}.assumed.role.credentials.provider",
                "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider"
            ),
        ]

        return GlueJobUtil.create_contexts(
                                app_env=app_env, 
                                aws_region_account_mappings=aws_region_account_mappings, 
                                bucket_prefix=bucket_prefix, 
                                assume_role_prefix=assume_role_prefix, 
                                kms_key_ssm_param_prefix=kms_key_ssm_param_prefix,
                                additional_spark_config=dataLakeConfig
                            )
    
    @staticmethod
    def create_logger(glue_context):
        return glue_context.get_logger()
    



In [ ]:
#Parametry
args=[]
args["JOB_NAME"] = "grsi-dp-save-recon-results-test"
args["AWS_REGION"] = "eu-west-1"
args["EU_ACCOUNT_NUMBER"] = "866525145675"
args["ENVIRONMENT"] = "test"
args["SOURCE_DATA_BUCKET"] = ""
args["INGESTION_BUCKET"] = "grsi-dp-ingestion"
args["ORGANIZATION_GUID"] = "2326c977-2f8d-4d97-8f13-a7ca2fd5d5db"
args["LM_TROUX_UID"] = "B3439EB1-509F-4193-B1B3-2023D75CBD29"
args["DEPLOYMENT_GUID"] = "489c81ee-7597-48ed-9526-d5164d7e01ff"
args["ASSUME_ROLE_PREFIX"] = "grsi-dp-glue-assume-ssm-role"
args["KMS_KEY_SSM_PARAM_PREFIX"] = "grsi-dp-extract-kms-param"
args["US_ACCOUNT_NUMBER"] = ""
args["EU_ACCOUNT_NUMBER"] = ""
args["DYNAMO_COLUMN_MAPPING_TABLE_NAME_PREFIX"] = ""
args["GLUE_SCRIPT_BUCKET"] = "grsi-dp-glue-script-eu-test"
args["XRAY_ECS_RESOURCE_ARN"] = ""


aws_regional_settings = {
    DataRegion.US: {"aws_region": "us-east-1", "account_number": args["US_ACCOUNT_NUMBER"]},
    DataRegion.EUROPE: {"aws_region": "eu-west-1", "account_number": args["EU_ACCOUNT_NUMBER"]},
    DataRegion.APAC: {"aws_region": "ap-southeast-2", "account_number": args["EU_ACCOUNT_NUMBER"]},
}



In [ ]:
# Create Contexts
# (
#     spark_context,
#     glue_context,
#     spark_session,
# ) = GlueJobUtil.create_contexts(
#     app_env=args["ENVIRONMENT"],
#     aws_region_account_mappings= {args["AWS_REGION"] : args["EU_ACCOUNT_NUMBER"] }
#     #{region_info.get("aws_region"): region_info.get("account_number") for data_region, region_info in aws_regional_settings.items()},        
#     bucket_prefix=args["INGESTION_BUCKET"],
#     assume_role_prefix=args["ASSUME_ROLE_PREFIX"],
#     kms_key_ssm_param_prefix=args["KMS_KEY_SSM_PARAM_PREFIX"]
# )
# logger = GlueLogger(GlueJobUtil.create_logger(glue_context))
# logger.info("GlueContext has been created successfully.")




In [ ]:
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

job = Job(glueContext)
job.init(args["JOB_NAME"], args)

